# Strands Agents と AgentCore Memory（ツール経由の長期記憶）

## はじめに

このノートブックでは、AgentCore Memory と Strands フレームワークを使用して、**共有長期記憶を持つマルチエージェントシステム**を実装する方法を示します。複数の特化エージェントが、各エージェント専用のネームスペースを持つ共通の長期記憶ストアにアクセスしながら連携する方法を探ります。

### チュートリアルの詳細

| 項目              | 詳細                                                                             |
|:------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | 長期会話型                                                                       |
| エージェントタイプ  | 旅行予約アシスタント                                                             |
| エージェントフレームワーク | Strands Agents                                                              |
| LLM モデル         | Anthropic Claude Haiku 4.5                                                      |
| チュートリアルコンポーネント | AgentCore User Preferences Memory 抽出、Memory の保存と取得のためのツール    |
| 例の複雑さ         | 中級                                                                             |

学習内容:

- 長期記憶戦略を使用した共有メモリリソースのセットアップ方法
- 専用メモリネームスペースにアクセスする特化エージェントの作成
- 特化エージェントに委譲するコーディネーターエージェントの実装
- エージェント特化のための構造化メモリネームスペースの活用

## シナリオコンテキスト

この例では、以下を含む**旅行計画システム**を作成します:
1. 旅行の設定と履歴の長期記憶を持つフライト予約アシスタント
2. 宿泊の設定の長期記憶を持つホテル予約アシスタント
3. これらの特化エージェントを調整するトラベルコーディネーター

各特化エージェントは共通のメモリストア内の独自のネームスペースにアクセスし、ユーザーの好みを時間をかけて永続的に理解できるようにします。このアプローチは、複雑なドメインをメモリインフラストラクチャを共有しながら独自の専門分野を維持する特化エージェントに分割する方法を示します。

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件

このチュートリアルを実行するには以下が必要です:
- Python 3.10+
- Amazon Bedrock AgentCore Memory 権限を持つ AWS 認証情報
- Amazon Bedrock AgentCore SDK

共有長期記憶リソースの環境セットアップと作成から始めましょう!

## ステップ 1: 環境セットアップ
このノートブックを動作させるために必要なすべてのライブラリをインポートし、クライアントを定義することから始めましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
import time
from datetime import datetime
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("travel-assistant")

Amazon Bedrock モデルと AgentCore の適切な権限を持つリージョンを定義します

In [ ]:
region = "us-west-2" # Replace with your AWS region
MODEL_ID= "global.anthropic.claude-haiku-4-5-20251001-v1:0"

## ステップ 2: 共有メモリリソースの作成
このセクションでは、各エージェント専用のネームスペースを持つ共通の長期記憶ストアを作成します。

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

client = MemoryClient(region_name=region)
memory_name = "TravelAgent_LTM"
memory_id = None

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory with Long-Term Strategy...")
    # Create the memory resource with a single long-term memory strategy
    # The {actorId} placeholder will be dynamically replaced with the actual actor ID
    memory = client.create_memory_and_wait(
        name=memory_name,
        description="Travel Agent with Long-Term Memory",
        strategies=[{
            StrategyType.USER_PREFERENCE.value: {
                "name": "UserPreferences",
                "description": "Captures user preferences",
                "namespaces": ["travel/{actorId}/preferences"]
            }
        }],
        event_expiry_days=7,  # Short-term conversation expires after 7 days
        max_wait=300,
        poll_interval=10
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### 長期記憶戦略の理解

作成するメモリリソースは、単一のユーザー設定戦略を使用した AgentCore Memory の長期記憶機能を使用します:

1. **ユーザー設定メモリ戦略**: 会話で言及されたユーザー設定を自動的に抽出して統合
2. **アクターベースのネームスペース**: ネームスペースパスにアクター ID を使用して、各エージェント用の別々のスペースを作成
3. **メモリの永続性**: 有効期限のある短期記憶とは異なり、抽出された設定は会話の有効期限を超えて永続化

ネームスペースパターン `travel/{actorId}/preferences` により、各特化エージェントはアクター ID に基づいて独自のネームスペースを持ちます:
- フライトエージェントがアクセス: `travel/flight-user-TIMESTAMP/preferences`
- ホテルエージェントがアクセス: `travel/hotel-user-TIMESTAMP/preferences`

これにより、各エージェントは共通のメモリインフラストラクチャを使用しながら、独自の専門知識を維持できます。

### エージェント ID のセットアップ

In [ ]:
# Create unique actor IDs for each specialized agent but share the session ID
flight_actor_id = f"flight-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
hotel_actor_id = f"hotel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"
flight_namespace = f"travel/{flight_actor_id}/preferences"
hotel_namespace = f"travel/{hotel_actor_id}/preferences"

In [ ]:
# Import the necessary components
from strands import Agent, tool
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider

### ステップ 3: メモリフックプロバイダーの作成

このステップでは、メモリ操作を自動化するカスタム `MemoryHookProvider` クラスを定義します。フックは、エージェントの実行ライフサイクルの特定のポイントで実行される特別な関数です。作成するメモリフックは 1 つの主要な機能を提供します:

1. **メモリの保存**: エージェントが応答した後に新しい会話を保存

これにより、手動管理なしでシームレスなメモリ体験が実現されます。

In [ ]:
class MemoryHookProvider(HookProvider):
    """Hook provider for automatic memory management"""
    
    def __init__(self, memory_id: str, client: MemoryClient):
        self.memory_id = memory_id
        self.client = client
    
    def save_memories(self, event: AfterInvocationEvent):
        """Save conversation after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2:
                # Get last user and assistant messages
                user_msg = None
                assistant_msg = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break
                
                if user_msg and assistant_msg:
                    # Get session info from agent state
                    actor_id = event.agent.state.get("actor_id")
                    session_id = event.agent.state.get("session_id")
                    
                    if not actor_id or not session_id:
                        logger.warning("Missing actor_id or session_id in agent state")
                        return
                    
                    # Save conversation
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                    )
                    logger.info("Saved conversation to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save memories: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks"""
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        logger.info("Memory hooks registered")

### メモリアクセスを持つ特化エージェントの作成

次に、特化エージェント用のシステムプロンプトを定義します。

In [ ]:
# ホテル予約スペシャリスト用のシステムプロンプト
HOTEL_BOOKING_PROMPT = f"""あなたはホテル予約アシスタントです。顧客がホテルを見つけ、予約し、宿泊施設やアメニティに関する質問に回答するのを支援します。
空室状況、料金、予約手順について明確な情報をフレンドリーで親切な方法で提供してください。1ターンにつき最大2つの質問をしてください。メッセージは短くし、顧客を圧倒しないでください。"""

# フライト予約スペシャリスト用のシステムプロンプト
FLIGHT_BOOKING_PROMPT = f"""あなたはフライト予約アシスタントです。顧客がフライトを見つけ、予約し、航空会社、ルート、旅行ポリシーに関する質問に回答するのを支援します。
フライトの空き状況、料金、スケジュール、予約手順について明確な情報をフレンドリーで親切な方法で提供してください。1ターンにつき最大2つの質問をしてください。メッセージは短くし、顧客を圧倒しないでください。"""

### エージェントツールの実装
コーディネーターエージェントが使用できるツールとして特化エージェントを実装します:

In [ ]:
@tool
def flight_booking_assistant(query: str) -> str:
    """
    Process and respond to flight booking queries.

    Args:
        query: A flight-related question about bookings, schedules, airlines, or travel policies

    Returns:
        Detailed flight information, booking options, or travel advice
    """
    try:
        provider_flight = AgentCoreMemoryToolProvider(
            memory_id=memory_id,      # Required
            actor_id=flight_actor_id, # Required
            session_id=session_id,    # Required
            region = region,
            namespace=flight_namespace
        )
        
        flight_memory_hooks = MemoryHookProvider(memory_id, client)

        flight_agent = Agent(
            tools=provider_flight.tools,
            hooks=[flight_memory_hooks],
            model=MODEL_ID,
            system_prompt=FLIGHT_BOOKING_PROMPT,
            state={"actor_id": flight_actor_id, "session_id": session_id}
        )

        # Call the agent and return its response
        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in flight booking assistant: {str(e)}"

@tool
def hotel_booking_assistant(query: str) -> str:
    """
    Process and respond to hotel booking queries.

    Args:
        query: A hotel-related question about accommodations, amenities, or reservations

    Returns:
        Detailed hotel information, booking options, or accommodation advice
    """
    try:
        provider_hotel = AgentCoreMemoryToolProvider(
            memory_id=memory_id,      
            actor_id=hotel_actor_id, 
            session_id=session_id,   
            region = region,
            namespace=hotel_namespace
        )

        hotel_memory_hooks = MemoryHookProvider(memory_id, client)

        hotel_booking_agent = Agent(
            tools=provider_hotel.tools,
            hooks=[hotel_memory_hooks],
            model=MODEL_ID,
            system_prompt=HOTEL_BOOKING_PROMPT,
            state={"actor_id": hotel_actor_id, "session_id": session_id}
        )
        
        # Call the agent and return its response
        response = hotel_booking_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in hotel booking assistant: {str(e)}"

### コーディネーターエージェントの作成

最後に、これらの特化ツールを調整するメインの旅行計画エージェントを作成します:

In [ ]:
# コーディネーターエージェント用のシステムプロンプト
TRAVEL_AGENT_SYSTEM_PROMPT = """
あなたは特化ツール間を調整する包括的な旅行計画アシスタントです:
- フライト関連のクエリ（予約、スケジュール、航空会社、ルート）には → flight_booking_assistant ツールを使用
- ホテル関連のクエリ（宿泊施設、アメニティ、予約）には → hotel_booking_assistant ツールを使用
- 完全な旅行パッケージには → 必要に応じて両方のツールを使用して包括的な情報を提供
- 一般的な旅行アドバイスや簡単な旅行の質問には → 直接回答

各エージェントは、ユーザーが履歴データについて尋ねた場合に備えて独自のメモリを持っています。
複雑な旅行リクエストを処理する際は、両方のツールからの情報を調整してまとまりのある旅行計画を作成してください。
複数のソースからの情報を提示する際は、明確な構成を提供してください。
1ターンにつき最大2つの質問をしてください。メッセージは短くし、顧客を圧倒しないでください。
"""

In [ ]:
travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

## マルチエージェントメモリシステムのテスト

旅行計画シナリオでマルチエージェントシステムをテストしましょう:

### オプション: フライト予約アシスタントの長期記憶をハイドレート
フライト予約エージェントをハイドレートしたい場合は、以下のセルのコメントを解除してください。

In [ ]:
"""flight_previous_messages = [
    ("Hi, I'm Sarah", "USER"),
    ("Hello Sarah! Welcome to FlightAssist. How can I help you with your travel plans today?", "ASSISTANT"),
    ("I'm looking to book a flight from New York to London sometime next month.", "USER"),
    ("I'd be happy to help you find flights from New York to London for next month. Do you have specific dates in mind, or are you flexible?", "ASSISTANT"),
    ("I'm thinking around the 15th to the 25th, but I can be a bit flexible.", "USER"),
    ("Great! That gives us some room to find the best options. Do you have any preferences regarding airlines or flight times?", "ASSISTANT"),
    ("I definitely prefer direct flights if possible. I really don't like layovers.", "USER"),
    ("I completely understand your preference for direct flights. Layovers can be inconvenient. Fortunately, there are several airlines offering direct flights between New York and London, including British Airways, American Airlines, Delta, and Virgin Atlantic.", "ASSISTANT"),
    ("That's good to hear. I've had good experiences with British Airways in the past.", "USER"),
    ("British Airways does offer excellent service on transatlantic routes. I'll keep that in mind when searching for options. Do you have any seating preferences or other requirements for your flight?", "ASSISTANT"),
    ("I always try to get an aisle seat. I like being able to get up without disturbing others, especially on long flights.", "USER"),
    ("An aisle seat is a great choice for long-haul flights like New York to London. I'll note your preference for aisle seating. Would you prefer to fly in the morning, afternoon, or evening?", "ASSISTANT"),
    ("I prefer overnight flights for long journeys. It helps me adjust to the time difference better.", "USER"),
    ("Overnight flights are indeed a smart choice for eastbound transatlantic travel. They allow you to arrive in London in the morning and help minimize jet lag. British Airways, Delta, and American all offer evening departures from New York that arrive in London the next morning.", "ASSISTANT"),
    ("Perfect! And I'm also wondering about baggage allowances since I'll be staying for about a week.", "USER"),
    ("For a week-long trip, most travelers find that a standard checked bag plus a carry-on is sufficient. British Airways typically allows one free checked bag on transatlantic flights in economy class, plus a carry-on and personal item. Would you like me to check the specific allowances for your preferred dates?", "ASSISTANT")
]

print("\nHydrating memories with previous conversations...")

# Save the conversation history to short-term memory
initial = client.create_event(
    memory_id=memory_id,
    actor_id=flight_actor_id,
    session_id=session_id,
    messages=flight_previous_messages,
)
print("✓ Conversation saved in short term memory")"""

In [ ]:
travel_agent("Hello, I would like to book a trip from LA to Madrid. From July 1 to August 2.")

In [ ]:
travel_agent("I prefer direct flights with Iberia")

In [ ]:
travel_agent("I would like a flight in the morning, in economy")

In [ ]:
travel_agent("I would like to fly from SNA, and return 15 days later")

## メモリの永続性テスト

メモリシステムが正しく動作しているかどうかをテストするために、旅行エージェントの新しいインスタンスを作成し、以前に保存された情報にアクセスできるかどうかを確認します:

In [ ]:
time.sleep(60) # Let's give the memory some time to process the events..
# Create a new instance of the travel agent
new_travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

# Ask about previous conversations
new_travel_agent("Can you remind me about flight preferences?")

## まとめ

このノートブックでは、以下のことをデモンストレーションしました:

1. 複数のエージェント用の共有メモリリソースの作成方法
2. メモリアクセスを持つツールとしての特化エージェントの実装方法
3. 会話コンテキストを維持しながら複数のエージェント間を調整する方法
4. 異なるエージェントインスタンス間でメモリが永続化される仕組み

共有メモリを持つこのマルチエージェントアーキテクチャは、専門ドメインを処理しながら一貫したユーザー体験を維持できる複雑な会話型 AI システムを構築するための強力なアプローチを提供します。

## クリーンアップ
このノートブックで使用したリソースをクリーンアップするためにメモリを削除しましょう。

In [ ]:
# client.delete_memory_and_wait(
#        memory_id = memory_id,
#        max_wait = 300,
#        poll_interval =10
#)